#Тегирование с помощью pymrphy2 и вероятностной модели снятия омонимии
@Василий Гурьев

#Imports

In [0]:
!pip install pymorphy2[fast]

     |████████████████████████████████| 51kB 1.9MB/s 
     |████████████████████████████████| 7.1MB 9.0MB/s 
     |████████████████████████████████| 378kB 45.6MB/s 
  Created wheel for DAWG: filename=DAWG-0.8.0-cp36-cp36m-linux_x86_64.whl size=859192 sha256=efce2c0a92a9ab12c008a665e1419867b4863197f99d754d900020f053ea9da2
  Stored in directory: /root/.cache/pip/wheels/3d/1f/f0/a5b1f9d02e193c997d252c33d215f24dfd7a448bc0166b2a12
Successfully built DAWG


In [0]:
import re
import numpy as np
import pandas as pd
import pymorphy2 # морфологический анализатор

from google.colab import drive
import os

from IPython.display import clear_output

#Defs

##text_to_sentences

In [0]:
def text_to_sentences(text): # режем текст на предложения
  return re.findall(r'["А-ЯЁA-Z].+?(?:["a-zа-яё]{2}|["A-ZА-ЯЁ]{2})(?:\.+|\!+|\?+)', text)

##sentence_to_words

In [0]:
def sentence_to_words(sentence): # режем предложения на слова
  return re.findall(r'[A-Za-zА-ЯЁа-яё0-9-]+', sentence)

##words_to_tags

In [0]:
def words_to_tags(words): # определение тегов слов в предложении
  tags = Dehomonym() # сниматель омонимии
  for word in words: # последовательно добавляем слова в обработку
    tags.add(word)
  tags.end() # добавляем токен окончания предложения
  return tags.paths # возвращаем цепочку тегов

##Dehomonym

In [0]:
class Dehomonym(): # класс для снятия омонимии

  def __init__(self):
    self.tags = [['START']] # тег начала предложения
    self.scores = [1] # вероятностная оценка тега начала предложения 
    # расчитывается перемножением вероятностей тегов из pymorphy2 с частотой их совместной встречаемости
    self.paths = [['START']] # начало цепочки тегов
    self.k_score = 1e+3 # коэффициент для замедления роста оценки с увеличением длины предложения
    self.on = True # снятие омонимии включено
    

  def word_to_parse(self, word): # получение списка вариантов морфологического анализа для слова
    if word not in parsed: # проверяем есть ли слово в обработанных
      parsed[word] = morph.parse(word) # получаем список вариантов морфологических анализа (объектов Parse)
    return parsed[word] # возвращаем список объектов Parse


  def get_occurrence(self, tag, new_tag): # получение частоты последовательности тегов
    '''
    тег - это список граммем; т.о. частота последовательности тегов - это минимальная
    частота последовательности граммем из данных тегов
    '''
    return min([dispositions.loc[left, right][0] for right in new_tag for left in tag])


  def add(self, word): # добавление вариантов тегов данного токена
    if self.on:
      parses = self.word_to_parse(word) # получаем список вариантов морфологического анализа для данного токена
      new_tags = [[grammeme for grammeme in grammemes if grammeme in parse.tag] for parse in parses]
      # извлекаем теги из объектов Parse и упаковываем их в список списков
      n = len(new_tags) # определяем количество вариантов анализа
      if len(new_tags[0]): # если результат анализа не пуст
        new_scores = [0] * n # создаём список вероятностных оценок
        new_paths = [[new_tags[i]] for i in range(n)] # создаём список вариантов окончаний цепочки
        for right, new_tag in enumerate(new_tags): # перебираем варианты тегов данного токена
          for left, tag in enumerate(self.tags): # перебираем вариенты тегов предыдущег токена
            score = self.scores[left] * parses[right].score * self.get_occurrence(tag, new_tag) / self.k_score
            # считатем оценку, перемножая оценку предыдущего тега, вероятность данного и частоту их встречаемости
            if score >= new_scores[right]: # если оценка больше предыдущей
              new_scores[right] = score # обновляем её
              path = self.paths[left] # и запоминаем цепочку, конец которой имеет максимальную оценку
          new_paths[right].extend(path) # добавляем её к вариантам тегов данного токена
        self.tags = new_tags # обновляем список вариантов тегов последнего токена
        self.scores = new_scores # обновляем список оценок вариантов тегов
        self.paths = new_paths # обновляем цепочки тегов
      else:
        self.on = False
        self.tags = None
        self.scores = None
        self.paths = None


  def end(self): # конец предложения
    if self.on:
      new_score = 0 # начаотная оценка конца предложения
      new_paths = [['END']] # сепочка с токеном конца предложения
      for left, tag in enumerate(self.tags): # перебираем вариенты тегов предыдущег токена
        score = self.scores[left] * self.get_occurrence(tag, ['END']) / self.k_score
        # считатем оценку
        if score >= new_score: # если оценка больше предыдущей
          new_score = score # обновляем её
          path = self.paths[left] # и запоминаем цепочку, конец которой имеет максимальную оценку
      new_paths[0].extend(path) # добавляем её к тегу конечного токена
      self.tags = [['END']] # обновляем список вариантов тегов последнего токена
      self.scores = [new_score] # бновляем список оценок вариантов тегов
      self.paths = [tag for tag in new_paths[0][-2:0:-1]] # разворачиваем цепочку, отбрасывая тег начального токена
      self.on = False # отключаем возможность добавлять токены
    # print(self.paths)

#Loading

In [0]:
morph = pymorphy2.MorphAnalyzer() # морфологический анализатор
parsed = {} # словарь обработанных токенов

In [0]:
drive.mount('/content/gdrive') # подключаем диск
os.chdir('/content/gdrive/My Drive/Colab Notebooks/NNDW') # задаём рабочую директорию

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive


In [0]:
dispositions = pd.read_csv('dicts/dispositions.csv', index_col=[0, 1]) # загрузка частот последовательности граммем
dispositions.tail()

,,occurrences
start,end,
Geox,END,59
Supr,END,0
Apro,END,5975
Cmp2,END,392
Init,END,605


In [0]:
grammemes = sorted(list(set(dispositions.index.get_level_values(0)) - {'START'})) # словарь граммем

#Parsing

In [0]:
for year in range(2002, 2003): # для каждого года (одного достаточно)
  sentences = pd.DataFrame(columns=['sentence']) # создаём коллекцию предложений
  data = pd.read_csv(f'data/newsru_{year}.csv', usecols=['text']) # загружаем данные
  data.drop_duplicates(inplace=True) # удаляем дубли
  data.text = data.text.apply(lambda s: re.sub(r'<p>|<s>|</p>|</s>', '', s)) # удаляем теги
  for i, text in enumerate(data.text): # для каждого текста
    print(year, i, end=' ') # печатаем год и номер текста
    sentences = pd.concat(
        [
        sentences,
        pd.DataFrame(text_to_sentences(text), columns=['sentence']) # режем текст на предложения
        ], ignore_index=True
        ) # и добавляем их в коллекцию предложений
    # clear_output()
  sentences.drop_duplicates(inplace=True) # удавляем дубли
  sentences['words'] = sentences.sentence.apply(sentence_to_words) # получаем списки слов для каждого предложения
  pd.DataFrame(columns=['tags', 'lemmas']).to_csv(f'data/tags_{year}.csv', index=False) # сохраняем будущую коллекцию тегов
  sentences.to_csv(f'data/sentences_{year}.csv', index=False) # сохраняем коллекцию предложений


for year in range(2002, 2003): # для каждого года
  words = pd.read_csv(f'data/sentences_{year}.csv', usecols=['words']) # загружаем коллекцию предложений как списка слов
  tags = pd.read_csv(f'data/tags_{year}.csv') # загружаем коллекцию тегов
  while len(words) > len(tags): # пока не получены теги для всех слов
    print(f'{year}: {len(tags)}/{len(words)}') # печатаем прогресс
    tags.loc[len(tags), 'tags'] = words_to_tags(eval(words.loc[len(tags), 'words']))
    # получаем список тегов из списка слов (который в Pandas хранится, как строка)

    if not len(tags) % 1000:
      tags.to_csv(f'data/tags_{year}.csv', index=False) # сохраняем промежуточный результат
  tags.to_csv(f'data/tags_{year}.csv', index=False) # сохраняем коллекцию тегов

0/173402


In [0]:
tags = pd.read_csv('data/tags_2002.csv') # изначально получал и леммы, но это накладно, и они не понадобятся
tags.head()

,tags,lemmas
0,"[['CONJ'], ['3per', 'VERB', 'impf', 'indc', 'p...","['как', 'сообщать', 'нтв', 'заокеанский', 'мед..."
1,"[['PREP'], ['ADJF', 'Apro', 'accs', 'neut', 's...","['по', 'их', 'слово', 'полезный', 'вещество', ..."
2,"[['NOUN', 'inan', 'masc', 'nomn', 'sing'], ['P...","['поход', 'в', 'сауна', 'как', 'средство', 'бо..."
3,"[['ADVB'], ['3per', 'NPRO', 'accs', 'masc', 's...","['там', 'он', 'иногда', 'называть', 'русский']"
4,"[['VERB', 'impr', 'intr', 'perf', 'plur'], ['3...","['запить', 'они', 'два', 'большой', 'стакан', ..."


In [0]:
tags.tags[0] # так выглядит список тегов

"[['CONJ'], ['3per', 'VERB', 'impf', 'indc', 'pres', 'sing', 'tran'], ['NOUN', 'inan', 'neut', 'nomn', 'sing'], ['ADJF', 'nomn', 'plur'], ['NOUN', 'anim', 'masc', 'nomn', 'plur'], ['3per', 'VERB', 'futr', 'indc', 'perf', 'plur', 'tran'], ['ADJF', 'Apro', 'datv', 'plur'], ['NOUN', 'anim', 'datv', 'masc', 'plur'], ['PRTF', 'actv', 'datv', 'past', 'perf', 'plur', 'tran'], ['PREP'], ['ADJF', 'ablt', 'masc', 'sing'], ['NOUN', 'ablt', 'inan', 'masc', 'sing'], ['INFN', 'perf', 'tran'], ['ADVB'], ['NOUN', 'accs', 'femn', 'inan', 'sing'], ['ADJF', 'gent', 'masc', 'sing'], ['NOUN', 'gent', 'inan', 'masc', 'sing']]"

In [0]:
texts = pd.read_csv(f'data/sentences_2002.csv') # а так коллекция предложений
texts.head()

,sentence,words
0,"Как сообщает НТВ, заокеанские медики рекоменду...","['Как', 'сообщает', 'НТВ', 'заокеанские', 'мед..."
1,"По их словам, полезные вещества в нем помогают...","['По', 'их', 'словам', 'полезные', 'вещества',..."
2,Поход в сауну как средство борьбы с похмельем ...,"['Поход', 'в', 'сауну', 'как', 'средство', 'бо..."
3,Там его иногда называют русским.,"['Там', 'его', 'иногда', 'называют', 'русским']"
4,Запейте их двумя большими стаканами воды.,"['Запейте', 'их', 'двумя', 'большими', 'стакан..."
